In [ ]:
# This will test SQL lib

In [14]:
import sqlglot

query = """
WITH ny AS (
    SELECT latitude AS latNY, longitude AS longNY
    FROM location
    WHERE city = 'NYC'
), 
aroundNY AS (
    SELECT l.city, l.latitude, l.longitude, e.NumberNegativeNews
    FROM location l
    JOIN event e ON e.locationID = l.locationID
    JOIN date d ON e.dateID = d.dateID
    CROSS JOIN ny
    WHERE d.fullDate = '2017-07-28'
)
SELECT * FROM aroundNY;
 """

# Parseando a query e convertendo para o dialeto PostgreSQL
parsed_query = sqlglot.parse_one(query)

parsed_query

Select(
  expressions=[
    Star()],
  from=From(
    this=Table(
      this=Identifier(this=aroundNY, quoted=False))),
  with=With(
    expressions=[
      CTE(
        this=Select(
          expressions=[
            Alias(
              this=Column(
                this=Identifier(this=latitude, quoted=False)),
              alias=Identifier(this=latNY, quoted=False)),
            Alias(
              this=Column(
                this=Identifier(this=longitude, quoted=False)),
              alias=Identifier(this=longNY, quoted=False))],
          from=From(
            this=Table(
              this=Identifier(this=location, quoted=False))),
          where=Where(
            this=EQ(
              this=Column(
                this=Identifier(this=city, quoted=False)),
              expression=Literal(this=NYC, is_string=True)))),
        alias=TableAlias(
          this=Identifier(this=ny, quoted=False))),
      CTE(
        this=Select(
          expressions=[
            Column(


In [29]:
# Creating expressions 
import sqlglot
from sqlglot.expressions import Func

# Criando a expressão personalizada para CLUSTER()
class Cluster(Func):
    arg_count = 1  # Quantidade de argumentos esperada

    def execute(self, context=None):
        # Aqui você pode chamar uma função Python específica quando o comando for executado
        return f"Executando função Python com {self.this.sql()}"

In [32]:
# Creating dialect

from sqlglot import exp
from sqlglot.dialects.dialect import Dialect
from sqlglot.generator import Generator
from sqlglot.tokens import Tokenizer, TokenType
from sqlglot import exp, generator, parser, tokens, transforms


class CustomEventMining(Dialect):
    
    class Tokenizer(tokens.Tokenizer):
        # Definindo CLUSTER como uma palavra-chave especial
        KEYWORDS = {**tokens.Tokenizer.KEYWORDS, "CLUSTER": "CLUSTER"}

    class Parser(parser.Parser):
        FUNCTIONS = {**parser.Parser.FUNCTIONS, "CLUSTER": Cluster.from_arg_list}

    class Generator(Generator):
        TRANSFORMS = {**Generator.TRANSFORMS, Cluster: lambda self, expr: f"CLUSTER({self.sql(expr.this)})"}


   

In [34]:
# Registrando o dialeto personalizado no sqlglot

# Query de teste
query = "SELECT CLUSTER(id) FROM users"

# Parseando com o novo dialeto
expression = sqlglot.parse_one(query, dialect=CustomEventMining)

# Exibindo a AST
print(expression.tree())

# Executando a função Python associada
cluster_expr = expression.find(Cluster)
if cluster_expr:
    print(cluster_expr.execute())  # Chama a função Python associada


ParseError: Invalid expression / Unexpected token. Line 1, Col: 14.
  SELECT [4mCLUSTER[0m(id) FROM users

## TESTE 2

In [35]:
import sqlglot
from sqlglot.expressions import Func, Table, Column
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

# Criando a expressão personalizada para CLUSTER()
class Cluster(Func):
    arg_count = 1  # Esperamos apenas uma coluna como argumento

    def execute(self, data):
        """
        Executa o algoritmo de clustering e retorna um DataFrame tabular.

        :param data: Lista de valores da coluna selecionada.
        :return: DataFrame representando os clusters.
        """
        if not isinstance(data, (list, np.ndarray, pd.Series)):
            raise ValueError("Os dados precisam ser uma lista ou array.")

        # Convertendo para array NumPy
        data = np.array(data).reshape(-1, 1)

        # Criando o modelo K-Means com 3 clusters
        kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
        clusters = kmeans.fit_predict(data)

        # Criando DataFrame com os resultados
        df_clusters = pd.DataFrame({
            "original_value": data.flatten(),
            "cluster": clusters
        })

        return df_clusters

    def to_cte(self, alias="resultados_cluster"):
        """
        Gera uma CTE a partir da execução do clustering.
        """
        df = self.execute(self.this)  # Executa o clustering
        columns = ", ".join(df.columns)  # Nome das colunas
        values = ", ".join(
            f"({', '.join(map(str, row))})" for row in df.values
        )  # Transformando valores para SQL

        # Criando a CTE
        return f"WITH {alias} AS (SELECT {columns} FROM (VALUES {values}) AS t({columns}))"

    def to_sql(self, dialect=None, **kwargs):
        """Converte para SQL"""
        return f"CLUSTER({self.this.sql(dialect=dialect, **kwargs)})"


In [8]:
from sqlglot import Dialect
from sqlglot.expressions import Func
from sqlglot.generator import Generator
from sqlglot.parser import Parser
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# Definindo a expressão CLUSTER()
class Cluster(Func):
    arg_count = 1  # Aceita apenas 1 argumento (coluna)


    def execute(self, data):
        """
        Executa o algoritmo de clustering e retorna um DataFrame tabular.

        :param data: Lista de valores da coluna selecionada.
        :return: DataFrame representando os clusters.
        """
        print(data)
        print(type(data))
        if not isinstance(data, (list, np.ndarray, pd.Series)):
            raise ValueError("Os dados precisam ser uma lista ou array.")

        # Convertendo para array NumPy
        data = np.array(data).reshape(-1, 1)

        # Criando o modelo K-Means com 3 clusters
        kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
        clusters = kmeans.fit_predict(data)

        # Criando DataFrame com os resultados
        df_clusters = pd.DataFrame({
            "original_value": data.flatten(),
            "cluster": clusters
        })

        return df_clusters
    
    def to_sql(self, dialect=None, **kwargs):
        """Define como a função CLUSTER será representada em SQL"""
        return f"CLUSTER({self.this.sql(dialect=dialect, **kwargs)})"

    def to_cte(self, alias="resultados_cluster"):
        """
        Gera uma CTE a partir da execução do clustering.
        """
        df = self.execute(self.this)  # Executa o clustering
        columns = ", ".join(df.columns)  # Nome das colunas
        values = ", ".join(
            f"({', '.join(map(str, row))})" for row in df.values
        )  # Transformando valores para SQL

        # Criando a CTE
        return f"WITH {alias} AS (SELECT {columns} FROM (VALUES {values}) AS t({columns}))"
# Criando um dialeto personalizado para reconhecer CLUSTER()
class CustomDialect(Dialect):
    class Parser(Parser):
        FUNCTIONS = {**Parser.FUNCTIONS, "CLUSTER": Cluster.from_arg_list}

    class Generator(Generator):
        TRANSFORMS = {**Generator.TRANSFORMS, Cluster: lambda self, expr: f"CLUSTER({self.sql(expr.this)})"}



In [9]:
import sqlglot


# Registrando o dialeto personalizado

# Query SQL simulada
query = """
WITH resultados_cluster AS (SELECT CLUSTER(price) FROM products)  
SELECT * FROM resultados_cluster;
"""

# Parseando a query
expression = sqlglot.parse_one(query, dialect=CustomDialect)

# Simulando dados (normalmente viriam do banco de dados)
dados_ficticios = [10, 20, 30, 200, 220, 250, 1000, 1100, 1200]

# # Encontrar a expressão CLUSTER na AST
cluster_expr = expression.find(Cluster)
cluster_expr.to_cte()
# Executar a função CLUSTER e gerar a CTE
# if cluster_expr:
#     cte_sql = cluster_expr.to_cte()
#     print(cte_sql)  # Exibindo a CTE gerada


price
<class 'sqlglot.expressions.Column'>


ValueError: Os dados precisam ser uma lista ou array.

In [10]:
import sqlglot
import pandas as pd
import numpy as np
from sqlglot import Dialect
from sqlglot.expressions import Func
from sqlglot.generator import Generator
from sqlglot.parser import Parser
from sklearn.cluster import KMeans


# Definindo a expressão CLUSTER()
class Cluster(Func):
    arg_count = 1  # Aceita apenas 1 argumento (coluna)

    def execute(self, df, column_name):
        if column_name not in df.columns:
            raise ValueError(f"A coluna '{column_name}' não foi encontrada no DataFrame.")

        data = df[column_name].values.reshape(-1, 1)
        # TODO fazer rodar com a minha classe do Kmenas
        # Criando o modelo K-Means com 3 clusters
        kmeans = KMeans(n_clusters=3, random_state=42, n_init="auto")
        clusters = kmeans.fit_predict(data)

        # Adicionando a coluna de cluster ao DataFrame
        df["cluster"] = clusters
        return df

    def to_cte(self, df, alias="resultados_cluster"):
        """
        Retorna um DataFrame representando a CTE gerada pelo CLUSTER.

        :param df: DataFrame contendo os dados.
        :param alias: Nome da CTE.
        :return: DataFrame transformado.
        """
        return self.execute(df, self.this.name)


# Criando um dialeto personalizado para reconhecer CLUSTER()
class CustomDialect(Dialect):
    class Parser(Parser):
        FUNCTIONS = {**Parser.FUNCTIONS, "CLUSTER": Cluster.from_arg_list}

    class Generator(Generator):
        TRANSFORMS = {
            **Generator.TRANSFORMS,
            Cluster: lambda self, expr: f"CLUSTER({self.sql(expr.this)})",
        }


# Simulando um DataFrame de entrada
df = pd.DataFrame({
    "id": range(1, 10),
    "price": [10, 20, 30, 200, 220, 250, 1000, 1100, 1200],
})

# Query SQL simulada
query = """
WITH resultados_cluster AS (SELECT CLUSTER(price) FROM df)  
SELECT * FROM resultados_cluster;
"""

# Parseando a query
expression = sqlglot.parse_one(query, dialect=CustomDialect)

# Encontrar a expressão CLUSTER na AST
cluster_expr = expression.find(Cluster)

# Executar a função CLUSTER e gerar um DataFrame com os resultados
df_clusterizado = cluster_expr.to_cte(df)

# Exibir o DataFrame final com os clusters
print(df_clusterizado)


   id  price  cluster
0   1     10        2
1   2     20        2
2   3     30        2
3   4    200        0
4   5    220        0
5   6    250        0
6   7   1000        1
7   8   1100        1
8   9   1200        1


# Exemplo com o SQLite

Daqui depois é so pegar a arvore construida e ir aplicando a execução dos itens



In [13]:
import sqlglot
import sqlite3
import numpy as np
import pandas as pd
from sqlglot import Dialect
from sqlglot.expressions import Func
from sqlglot.generator import Generator
from sqlglot.parser import Parser
from sklearn.cluster import KMeans


# Definindo a expressão CLUSTER()
class Cluster(Func):
    arg_count = 1  # Aceita apenas 1 argumento (coluna)

    def execute(self, data):
        """
        Executa o algoritmo de clustering e retorna um DataFrame tabular.

        :param data: Lista de valores da coluna selecionada.
        :return: DataFrame representando os clusters.
        """
        if not isinstance(data, (list, np.ndarray, pd.Series)):
            raise ValueError("Os dados precisam ser uma lista ou array.")

        # Convertendo para array NumPy
        data = np.array(data).reshape(-1, 1)

        # Criando o modelo K-Means com 3 clusters
        kmeans = KMeans(n_clusters=3, random_state=42, n_init="auto")
        clusters = kmeans.fit_predict(data)

        # Criando DataFrame com os resultados
        df_clusters = pd.DataFrame({"original_value": data.flatten(), "cluster": clusters})

        return df_clusters

    def to_cte(self, alias="resultados_cluster", data=None):
        """
        Gera uma CTE a partir da execução do clustering.

        :param alias: Nome da CTE.
        :param data: Lista de valores extraídos do banco de dados.
        :return: String SQL com a CTE gerada.
        """
        if data is None:
            raise ValueError("Os dados precisam ser fornecidos para a CTE.")

        df = self.execute(data)  # Executa o clustering
        columns = ", ".join(df.columns)  # Nome das colunas
        values = ", ".join(
            f"({', '.join(map(str, row))})" for row in df.values
        )  # Transformando valores para SQL

        # Criando a CTE
        return f"WITH {alias} AS (SELECT {columns} FROM (VALUES {values}) AS t({columns}))"


# Criando um dialeto personalizado para reconhecer CLUSTER()
class CustomDialect(Dialect):
    class Parser(Parser):
        FUNCTIONS = {**Parser.FUNCTIONS, "CLUSTER": Cluster.from_arg_list}

    class Generator(Generator):
        TRANSFORMS = {
            **Generator.TRANSFORMS,
            Cluster: lambda self, expr: f"CLUSTER({self.sql(expr.this)})",
        }


def executar_query_do_arquivo(nome_arquivo="output.txt"):
    """
    Lê um arquivo com uma query SQL-like, executa no SQLite e retorna o resultado como um DataFrame.
    """
    # Criando conexão com SQLite em memória
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()

    # Criando uma tabela fictícia para os testes
    cursor.execute("CREATE TABLE TABELA (DATA_ID INTEGER, DATA REAL);")
    cursor.executemany(
        "INSERT INTO TABELA (DATA_ID, DATA) VALUES (?, ?);",
        [(1, 10), (2, 20), (3, 30), (4, 200), (5, 220), (6, 250), (7, 1000), (8, 1100), (9, 1200)]
    )
    conn.commit()

    # Lendo a query do arquivo
    with open(nome_arquivo, "r") as f:
        query = f.read().strip()

    # Parseando a query para interceptar a função CLUSTER()
    expression = sqlglot.parse_one(query, dialect=CustomDialect)

    # Encontrando a expressão CLUSTER na AST
    cluster_expr = expression.find(Cluster)

    if cluster_expr:
        # Executando a query base para obter os dados reais
        base_query = "SELECT DATA FROM TABELA"
        df_base = pd.read_sql_query(base_query, conn)

        # Executando a função CLUSTER
        df_clusterizado = cluster_expr.to_cte(data=df_base["DATA"].tolist())

        # Criando uma tabela temporária para armazenar os clusters
        cursor.execute("CREATE TEMP TABLE CLUSTERS (original_value REAL, cluster INTEGER);")
        for _, row in df_clusterizado.iterrows():
            cursor.execute("INSERT INTO CLUSTERS (original_value, cluster) VALUES (?, ?);", (row["original_value"], row["cluster"]))

        conn.commit()

        # Executando a query final no SQLite
        df_final = pd.read_sql_query("SELECT cluster FROM CLUSTERS;", conn)

        # Fechando a conexão
        conn.close()

        return df_final
    else:
        raise ValueError("Nenhuma função CLUSTER encontrada na query!")


# Executar e imprimir o resultado
df_resultado = executar_query_do_arquivo()
print(df_resultado)


AttributeError: 'str' object has no attribute 'iterrows'